In [3]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

dmax = 30 # 搜寻匹配的范围
window_size = 2 # 匹配窗大小
f = 30 # 焦距
T = 20 # 两个相机间距
alpha = 1 # 阈值ve的系数

left = cv2.imread('view1m.png') # 读取左相机的图像
right = cv2.imread('view5m.png') # 读取右相机的图像
size1, size2 = left.shape[0], left.shape[1] # 获取图像的长和宽
# disparity用于存储视差图
disparity = np.zeros((size1, size2), dtype=np.uint8)
e = np.zeros_like(disparity) # 用于存储误差能量矩阵
e_avg = np.ones_like(disparity) # 用于存储平均误差能量矩阵
e_avg = e_avg * 100000 # 初始化为一个较大的数
# 遍历所有可能的视差值，求出以左图像(i,j)为中心，窗口大小为window_size*window_size的像素块的误差能量e(i,j,d)
for d in range(dmax):
    # 求出以左图像(i,j)为中心，窗口大小为window_size*window_size的像素块的误差能量e(i,j,d)
    for i in range(size1):
        for j in range(size2):
            sum = 0
            for m in range(window_size):
                for n in range(window_size):
                    for k in range(3):
                        # # 计算左右视图像素之间的差异
                        x = min(size1 - 1, i + m)# 取i+m和size1-1中较小的一个，防止越界
                        y = min(size2 - 1, j + n)# 取j+n和size2-1中较小的一个，防止越界
                        # 左图像素位置为(x, min(y + d, size2 - 1))，右图像素位置为(x, y)
                        square_diff = (int(left[x][min(y + d, size2 - 1)][k]) - int(right[x][y][k])) ** 2
                        sum = sum + square_diff
            # 计算误差能量
            e[i][j] = sum / (3 * window_size * window_size)

   # 计算平均误差能量e_avg(i,j,d)
    for i in range(size1):
        for j in range(size2):
            e_temp = 0
            for m in range(window_size):
                for n in range(window_size):
                    x = min(size1 - 1, i + m)
                    y = min(size2 - 1, j + n)
                    e_temp = e_temp + e[x][y]
            # 若e_avg(i,j,d)比之前计算的更小，则更新disparity(i,j)为当前的视差值d，并更新e_avg(i,j,d)
            e_temp = e_temp / (window_size * window_size)
            if e_temp < e_avg[i, j]:
                e_avg[i, j] = e_temp
                disparity[i, j] = d
# 将计算的视差图保存为disparity_base.png
cv2.imwrite('disparity_base.png', disparity)
# 读取disparity_base.png并将其转换为灰度图像
temp = cv2.imread('disparity_base.png')
gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY) 
# 对灰度图像应用直方图均衡化
dst = cv2.equalizeHist(gray)
# 将直方图均衡化的结果保存为disparity_base_enhanced.png
cv2.imwrite('disparity_base_enhanced.png', dst)
# 读取直方图均衡化后的disparity_base_enhanced.png
temp2 = cv2.imread('disparity_base_enhanced.png')
# 对视差图进行中值滤波
cv2.medianBlur(disparity, 3)
# 计算可靠度
ve = alpha * e_avg.mean()
count_not_ne = 0
sum_e = 0
for i in range(size1):
    for j in range(size2):
        if e_avg[i][j] > ve:
            disparity[i][j] = 0
        else:
            sum_e = sum_e + e_avg[i][j]
            count_not_ne = count_not_ne + 1
reliability = float(count_not_ne) ** (-1) * sum_e
print("可靠度：", reliability)
# 将过滤后的视差图保存为disparity_reliable.png
cv2.imwrite('disparity_reliable.png', disparity)
# 读取disparity_reliable.png并将其转换为灰度图像
temp = cv2.imread('disparity_reliable.png')
gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
# 对灰度图像应用直方图均衡化
dst = cv2.equalizeHist(gray)
# 将直方图均衡化的结果保存为disparity_reliable_enhanced.png
cv2.imwrite('disparity_reliable_enhanced.png', dst)
# 读取直方图均衡化后的disparity_reliable_enhanced.png
temp2 = cv2.imread('disparity_reliable_enhanced.png')


# 生成深度图像
depth = np.ones_like(disparity, dtype=np.uint8)
for i in range(size1):
    for j in range(size2):
        if disparity[i][j] < 5:  ##噪音
            depth[i][j] = 0
        else:
            depth[i][j] = f * T // disparity[i][j]
# 对深度图像进行中值滤波
cv2.medianBlur(depth, 3)
# 将深度图像保存为depth_reliable_medianblur.png
cv2.imwrite('depth_reliable_medianblur.png', depth)
# 读取depth_reliable_medianblur.png并将其转换为灰度图像
temp = cv2.imread('depth_reliable_medianblur.png')
gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)  
# 对灰度图像应用直方图均衡化
dst = cv2.equalizeHist(gray)
# 将直方图均衡化的结果保存为depth_reliable_medianblur_enhanced.png
cv2.imwrite('depth_reliable_medianblur_enhanced.png', dst)
temp2 = cv2.imread('depth_reliable_medianblur_enhanced.png')
# 定义X，Y轴为图像尺寸
X = range(size1)
Y = range(size2)

Z = depth
# 生成网格
xx, yy = np.meshgrid(X, Y)  
# 展开X和Y轴的网格为一维数组
X, Y = xx.ravel(), yy.ravel()  
# 柱状图底部置零
bottom = np.zeros_like(X) 
# # 将深度数据展开成一维数组
Z = Z.ravel()  
# 柱状图宽度和高度均为1
width = height = 1  
fig = plt.figure()
# 创建3D图像对象
ax = Axes3D(fig)
# 绘制3D柱状图
ax.bar3d(X, Y, bottom, width, height, Z, shade=True)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z(depth)')
plt.show()


可靠度： 10.70080287697583


<Figure size 640x480 with 0 Axes>